In [4]:
import spacy
nlp = spacy.load('de_core_news_md')

In [39]:
import pandas as pd
import os


In [ ]:

def parse_text_to_df(input_text):
    lines = input_text.splitlines()
    df = pd.DataFrame(columns=['Chapter', 'These', 'Text'])
    chapter = None
    these = ''
    text = ''

    for line in lines:
        if line.startswith("### "):  # Kapitel

            if chapter != line[4:].strip() and chapter != None:
                df.loc[len(df)] = [chapter, these, text.strip()]
                text = ''

            chapter = line[4:].strip()
            these = None #Rücksetzten der These bei neuem Kapitel

        elif line.startswith("## "):  #These

            if these != line[3:].strip():
                df.loc[len(df)] = [chapter, these, text.strip()]
                text = ''

            these = line[3:].strip()
        else:  # Kombinieren aufainanderfolgender Texte im Unterkapitel
            text += line[1:].strip()


    return df


In [61]:
file_name = 'all_parties_text_combined.parquet' 

if os.path.exists(file_name) == False:
    parties_file_list = ['spd.txt', 'cdu.txt', 'gruene.txt', 'afd.txt', 'fdp.txt', 'linke.txt']
    all_textes_df = pd.DataFrame()
    for party_file in parties_file_list:
        with open (party_file, 'r', encoding="utf-8") as f:
            text = f.read()

        df = parse_text_to_df(text)
        df['Party'] = party_file[:-4]

        all_textes_df = pd.concat([all_textes_df, df], ignore_index=True)

    all_textes_df = all_textes_df[['Party', 'Chapter', 'These', 'Text']]

    all_textes_df.to_parquet(file_name, index=False)

In [63]:
all_textes_df = df = pd.read_parquet("all_parties_text_combined.parquet")
all_textes_df.nunique()

Party        6
Chapter     58
These      492
Text       534
dtype: int64

In [64]:
all_textes_df['Party'].value_counts()

Party
gruene    145
fdp       124
spd        99
linke      85
afd        82
cdu        17
Name: count, dtype: int64

In [65]:
all_textes_df

,Party,Chapter,These,Text
0,spd,Worauf es jetzt ankommt,None,Am 23. Februar findet die vorgezogene Bundesta...
1,spd,Ein neuer Aufschwung für Deutschland,None,Viele Menschen machen sich derzeit Sorgen um d...
2,spd,1. Wir kämpfen für neues Wachstum und sichere ...,None,"Dies ist das erste, aber sicher nicht das letz..."
3,spd,1. Wir kämpfen für neues Wachstum und sichere ...,Wir wollen Energiepreise senken und zentrale E...,"Wir brauchen bezahlbare Energie, sonst geht un..."
4,spd,1. Wir kämpfen für neues Wachstum und sichere ...,"Wir wollen, dass Unternehmen die besten Rahmen...",Wir können etwas tun für unsere Unternehmen un...
...,...,...,...,...
547,linke,18. Inklusion und Teilhabe – selbstverständlich!,None,Nach unserem Verständnis ist Inklusion der Geg...
548,linke,19. Medien und Kultur für eine plurale Gesells...,None,Für demokratische Aushandlung und Meinungsbild...
549,linke,19. Medien und Kultur für eine plurale Gesells...,Kultur – vielfältig und für alle zugänglich,Der Zugang zu Kultur soll nicht vom Geldbeutel...
550,linke,19. Medien und Kultur für eine plurale Gesells...,Sport ist kein Luxus,Sport ist für alle da. Dafür müssen die Zugang...
